# CARGA DE LIBRERIAS

In [1]:
#%% CARGA DE LIBRERIAS
# = = = = = = = = = = =

# Función para intentar importar y, en caso de error, instalar y luego importar
# -----------------------------------------------------------------------------

def try_import(module_name, install_name=None, alias=None, fromlist=None):

    try:

        if fromlist:

            module = __import__(module_name, fromlist=fromlist)
            globals().update({name: getattr(module, name) for name in fromlist})

        else:
            module = __import__(module_name)

            if alias:

                globals()[alias] = module

            else:

                globals()[module_name] = module

    except ImportError:

        if install_name is None:

            install_name = module_name

        !pip install {install_name}

        if fromlist:

            module = __import__(module_name, fromlist=fromlist)

            globals().update({name: getattr(module, name) for name in fromlist})

        else:
            module = __import__(module_name)

            if alias:

                globals()[alias] = module

            else:

                globals()[module_name] = module



# Autenticación de Google Colab
# -----------------------------

try_import('gspread')
#try_import('google.colab', 'google-colab')
#try_import('google.auth', 'google-auth')

try_import('google.colab', fromlist=['auth'])
try_import('google.auth', fromlist=['default'])
try_import('google.auth.transport.requests', fromlist=['Request'])
try_import('google.oauth2.service_account', fromlist=['Credentials'])

# Librerías para gráficos y análisis de datos
# -------------------------------------------

try_import('dash')
try_import('plotly')
try_import('plotly.express', alias='px')
try_import('jupyter_dash', fromlist=['JupyterDash'])
try_import('matplotlib.pyplot', alias='plt')
try_import('numpy', alias='np')
try_import('pandas', alias='pd')

# Librerías adicionales
# ---------------------

try_import('textwrap')
try_import('threading', fromlist=['Timer'])
try_import('webbrowser')
try_import('os')

# Dash modules
# ------------

try_import('dash', fromlist=['dcc', 'html'])
try_import('dash.dependencies', fromlist=['Input', 'Output'])

import warnings

warnings.simplefilter(action='ignore', category=Warning)


print("Todas las librerías se han importado correctamente.")

Todas las librerías se han importado correctamente.


# DEFINICION DE FUNCIONES

## Graficadoras (para Dashboard)

In [2]:
'''
#%% FUNCION PARA TRABAJAR CON LOS TITULOS DE LOS PLOTS
# = = = = = = = = = = = = = = = = = = = = = = = = = = =

def wrap_title(text):
    text = '\n'.join(textwrap.wrap(text, width=150))
    # We do this in two steps so that newlines present before wrapping also get
    # converted to BRs
    return text.replace('\n', '<br>\n')


#%% FUNCION PARA CREAR GRAFICO DE BARRAS (PARA DASHBOARD)
# = = = = = = = = = = = = = = = = = = = = = = = = = = = =

def grf_subplots_barras(df, titulos_columnas, wrap_length=50):

    figuras = []

    for columna in titulos_columnas:

        # Calcular las frecuencias y los porcentajes
        # ------------------------------------------

        data = df[columna].value_counts().reset_index()
        data.columns = [columna, 'Frecuencia']
        data['Porcentaje'] = (data['Frecuencia'] / data['Frecuencia'].sum()) * 100

        # Verificar si los datos están vacíos
        # ------------------------------------

        if data.empty:

            print(f'La columna {columna} no tiene datos válidos.')

            continue

        # Envolver el título para que no se desborde
        # ------------------------------------------

        title=wrap_title(columna)

        # Crear la gráfica de barras con etiquetas personalizadas
        # ------------------------------------------

        fig = px.bar(
            data,
            x=columna,
            y='Frecuencia',
            text=data.apply(lambda row: f'{row["Frecuencia"]} ({row["Porcentaje"]:.1f}%)', axis=1)
        )

        # Actualizar las etiquetas de las barras
        # --------------------------------------

        fig.update_traces(textposition='inside')

        # Ajustar la apariencia del título
        # --------------------------------

        fig.update_layout(
            title={
                'text': title,
                'y':0.93,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            xaxis_title='',  # Eliminar el título del eje x
            yaxis_title='Frecuencia'
        )

        figuras.append(fig)

    return figuras



#%% FUNCION PARA CREAR GRAFICOS DE TORTA (PARA DASHBOARD)
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

def grf_torta(df, titulos_columnas):

    figuras = []

    for columna in titulos_columnas:

        fig = px.pie(df, names=columna, title=columna)
        figuras.append(fig)

    return figuras
''';

# PREGUNTAS POR SECCION

In [3]:
s_organizacion = ['Cuenta con un organigrama escrito',
                  'Posee definida y documentada una política de Calidad/Inocuidad',
                  'Cuenta con un manual de Calidad/Inocuidad',
                  'Cuenta con un sistema de gestión de la calidad implementado (Ej. ISO 9001)',
                  'Cuenta con un sistema de gestión de la calidad certificado',
                  'Tiene implementado las BPM',
                  'Tiene implementado HACCP',
                  'Si tiene implementado HACCP, adjunte el plan aquí',
                  'Cuenta con algún sistema de gestión de la inocuidad implementado (Ej. ISO 22000, esquema FSSC22000, BRC)',
                  'Cuenta con algún sistema de gestión de la inocuidad certificado',
                  'La alta dirección realiza revisiones periódicas',
                  'La alta dirección designó un líder del equipo de inocuidad',
                  'Nombre del líder del equipo de inocuidad',
                  'Cuenta con Personal Calificado para Controles Preventivos (PCQI)',
                  'Nombre del Personal Calificado para Controles Preventivos',
                  'Observaciones (Organización)'
                 ]

s_informacion_general = ['Cuenta con un programa de orden y limpieza documentado',
                         'Cuenta con un control médico e higiénico del personal',
                         'Cuenta con un programa de capacitación escrito y documentado',
                         'Cuenta con un programa de trazabilidad de productos y subproductos',
                         'Cuenta con procedimientos para gestionar potenciales situaciones de emergencia (crisis)',
                         'Cuenta con un programa de atención de reclamos',
                         'Cuenta con un programa de Recall',
                         'Cuenta con un análisis de vulnerabilidad (Fraude)',
                         'Se contempla el manejo de alérgenos para prevenir la contaminación cruzada. Estos son leche, huevos, pescados, crustáceos, trigo, soja, maní, nueces o almendras, ni tampoco sulfito de sodio',
                         'Los alimentos que contienen gluten se mantienen separados de los que son libres de gluten para prevenir la contaminación cruzada',
                         'Los insumos, productos intermedios o terminados no son derivados de Organismos Genéticamente Modificados (OGMs)',
                         'Observaciones (Información General)'
                        ]

s_personal = [
    'Cuenta el personal con la vestimenta adecuada para la tarea.',
    'Personal competente tiene asignada la responsabilidad para asegurar el cumplimiento de las BPM',
    'Ante condiciones de salud indeseables se toman medidas correctivas apropiadas'
    'Las instalaciones están provistas con lavamanos',
    'Cuenta con un plan anual escrito de capacitación en BPM e inocuidad alimentaria',
    'Las áreas de producción tienen prohibiciones respecto del uso de tabaco, bebidas, alimentos, etc.',
    'Observaciones (Personal)'
]


s_edificio_instalaciones = [
    'Se lleva a cabo un adecuado programa de control de plagas, sin madrigueras, nidos, etc. No existe pasto, malezas u otro tipo de vegetación que provoque la instalación de roedores y además posibilite el desarrollo de microorganismos e insectos. No existe acumulación de escombros, madera, etc. Los cercos y paredones están enteros sin roturas ni agujeros.',
    'Todas las puertas de acceso, ventanas están cerradas y contienen mallas para evitar el ingreso de insectos.',
    'Los pisos, paredes, y cielorrasos son construidos de materiales de fácil limpieza y mantenidos en buenas condiciones.',
    'No existen cañerías o superficies que condensan, suspendidas sobre áreas de trabajo donde pueden ser potenciales contaminantes.',
    'Existe separación de áreas para evitar contaminaciones cruzadas.',
    'Existe una adecuada iluminación en las áreas donde se examinan los insumos, áreas de proceso o almacenamiento, área de limpieza de equipos y utensilios, etc.',
    'Existe protección para las luminarias u otro tipo de elemento de vidrio suspendido para evitar contaminación física.',
    'No se observan partes metálicas oxidadas que pongan en riesgo una contaminación hacia el producto.',
    'Observaciones (Edificio instalaciones)'
    ]


s_equipamiento = [
    'El equipo es apropiado para el uso asignado y puede ser limpiado y mantenido adecuadamente.',
    'El diseño, construcción y uso del equipo previene de contaminación con lubricantes, combustibles, fragmentos metálicos, agua contaminada u otro contaminante.',
    'Observaciones (Equipamiento)'
]


s_manejo_de_materiales = [
    'Insumos y materiales de empaque son segregados adecuadamente del producto final hasta ser utilizados.',
    'Materiales peligrosos son almacenados y controlados adecuadamente para prevenir mal uso o hurtos. (fungicidas, ceras, etc. utilizados en tratamientos post cosechas)',
    'Observaciones (Manejo de materiales)'
]


s_instalaciones_sanitarias = [
    'Existe un adecuado control del agua utilizada en el proceso de producción, de acuerdo a regulaciones oficiales.',
    'El agua es controlada en todas las etapas del proceso para evitar contaminaciones hacia el producto y subproducto',
    'Existen sanitarios adecuados y mantenidos en buenas condiciones higiénicas todo el tiempo. Existen carteles que recomienden al personal higienizarse las manos con agua y jabón antes de retornar a su puesto de trabajo.',
    'Los desechos y residuos son retirados y transportados adecuadamente en tiempo y forma.',
    'Observaciones (Instalaciones sanitarias)'
    ]


s_almacenaje_y_distribucion = [
    'Todos los depósitos están cerrados y diseñados para máxima seguridad externa.',
    'Los materiales de empaque tales como etiquetas, precintos, y demás material impreso de identificación del producto se encuentran en un lugar apartado, en un área de acceso restringido.',
    'Los productos químicos tales como sanitizantes, pesticidas, productos para tratamiento de agua, se encuentran almacenados separados de los demás insumos.',
    'Todos los insumos son almacenados a la temperatura que indica el fabricante.',
    'Observaciones (Almacenaje y distribución)'
    ]

s_control_de_laboratorio = [
    'Cuenta con personal de laboratorio capacitado para el control de proceso.',
    'Se analizan materiales extraños y contaminantes en los insumos. Todos los que presenten desvíos son apropiadamente separados y rechazados.',
    'Se realizan controles ambientales rutinarios.',
    'Cuentan con un programa de calibración de los equipos de laboratorio.',
    'Realizan análisis en laboratorios externos',
    'Observaciones (Control de laboratorio)'
]


s_inocuidad = [
    'Se utilizan sanitizantes en el agua y se controla su concentración.'
    'Se aplican productos fitosanitarios teniendo en cuenta las regulaciones de los países de destino del producto y subproducto.',
    'Se controlan los niveles de residuos de los productos fitosanitarios.',
    'Observaciones (Inocuidad)'
]




s_food_defense = [
    'Cuenta con un programa de Food Defense',
    'Todas las puertas de acceso a zonas restringidas están cerradas.',
    'Los visitantes son identificados.',
    'Se le exige a los contratistas registrarse antes del ingreso.',
    'El personal de producción está identificado por medio de su vestimenta o con tarjetas de identificación.',
    'Observaciones (Food Defense)'
]


s_ambiente = [
    'Posee un sistema de gestión ambiental implementado.',
    'Posee control de efluentes/manejo de los residuos/relevamiento de actividades críticas/utiliza transporte de sustancias peligrosas.',
    'Observaciones (Ambiente)'
]


s_responda_las_siguientes_preguntas = [
    '¿La empresa tiene producción primaria?',
    '¿Se compra fruta a otros productores?',
    '¿Cuáles son los criterios de selección de dichos proveedores?',
 '¿Cuál es la capacidad de procesamiento de la planta? (En toneladas)',
 '¿Cuál es la capacidad de almacenamiento en frío? (En toneladas)',
 '¿Cómo se realiza el almacenamiento en frío?',
 '¿Cuáles son las variedades trabajadas de cada especie?',
 '¿Cuáles son los motivos de descarte cuantitativamente más importantes?',
 '¿Qué porcentajes de descarte son los habituales?',
 '¿Todos los lotes producidos de productos y subproductos son de la misma especie, evitando las mezclas de peras con manzanas u otras especies?',
 '¿Cuál es el sistema utilizado de codificación para productos y subproductos?',
 '¿Cuáles son las condiciones de almacenaje de productos y subproductos?',
 '¿Se utiliza la tecnología de MCP?',
 'Si la utiliza, ¿Para qué variedades?',
 '¿Aplica FIFO para la expedición de sus productos y subproductos?',
 'Detalle la documentación que acompaña al producto que le provee a Jugos S.A.',
 'Detalle los productos fitosanitarios que utiliza en el proceso productivo.',
 '¿Cuál es la fuente utilizada para estar actualizado respecto de los límites de tolerancia de los productos fitosanitarios utilizados según el país de destino?',
 '¿Conoce las pautas de entrega de materia prima?'
]


s_mapeo_de_cadena_de_suministro = [
     '¿Cómo describe la relación comercial con cliente (Jugos S.A.)?',
 '¿Está interesado en implementar proyectos colaborativos de mejora?',
 'Observaciones (Mapeo de cadena de suministro)',
 '¿Qué cantidad (en toneladas) suministra a Jugos SA en el año?'
]


s_perfil_proveedores_perfil = [
    '¿Participa de programas de auditorias sociales?',
 'Si la respuesta anterior fue si, indique los programas de auditorias sociales  en los que participa',
 'Adjunte la documentación aquí',
 '¿Qué distancia (en Km) hay entre planta de empaque y Jugos S.A.?'
]


s_perfil_proveedores_mano_de_obra = [
    '¿Qué cantidad de personal trabaja en la producción en temporada ALTA? (promedio)',
 '¿Qué cantidad de personal trabaja en la producción en temporada BAJA? (promedio)',
 '¿Se le solicita Libreta Sanitaria/carnet de manipulación de alimentos al personal al iniciar la temporada?',
 '¿Se conoce si el personal sufrió enfermedades infectocontagiosas durante los últimos doce meses?',
 '¿Se dispone de elementos de primeros auxilios en el lugar?',
 '¿El personal tiene definido el uso de Elementos de Protección Personal (EPP) según sus tareas?',
 '¿Qué porcentaje de mujeres (vs. hombres) trabajan en producción? En temporada ALTA',
 '¿Qué porcentaje de mujeres (vs. hombres) trabajan en producción? En temporada BAJA',
 '¿Posee información de condiciones de vida de los empleados?',
 '¿Cómo se transportan de casa al trabajo?',
 '¿Cuánto tiempo dura el viaje?',
 'Si viven muy lejos, ¿dónde duermen usualmente?',
 '¿Ha tenido problemas legales en cuanto a trabajos forzados, empleos ilegales, condiciones no adecuadas de trabajo, etc.? ',
 '¿Ha tenido problemas de pago de salarios?',
 '¿Cómo evalúa el mercado laboral en los próximos años? '
]



s_agua = [
     '¿De donde proviene el agua que utiliza?',
 '¿Mide y/o monitorea el consumo de agua en general?',
 '¿Considera que se halla en lugar donde el recurso agua es una restricción?',
 '¿Considera que la polución de agua es regular u ocasionalmente un problema en su área?'
]


s_uso_de_pesticidas = [
    '¿Se registra el uso de agroquímicos? (tipo de pesticida usado)',
 '¿Registra la cantidad dosificada de cada producto?',
 '¿Tiene conocimiento de qué pesticidas están permitidos?',
 '¿Cuál es la fuente?',
 '¿Qué tipo de técnica utiliza para dosificar estos productos?',
 '¿Se provee Equipo de Protección Personal (EPP) a responsable de manipulación de estos productos?',
 '¿Se provee entrenamiento al personal, y se registra?',
 '¿Revisa el uso de los EPP y registra?',
 '¿Tiene definido un plan de acción para minimizar el uso de pesticidas?',
 '¿Qué plan de acción tiene definido para minimizar el uso de pesticidas?'
]


s_residuos = [
     '¿Clasifican los residuos?',
 '¿Cuáles?',
 '¿Realiza gestión de residuos peligrosos?',
 '¿Cómo los gestiona?'
]

s_pauta_firmada = [
    'Subir pauta firmada aquí'
]

# CARGA DE DATOS

In [4]:
#%% OBTENCION CREDENCIALES PARA ACCESO A CUENTA
# = = = = = = = = = = = = = = = = = = = = = = = =

#scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
#creds = Credentials.from_service_account_file("massive-rock-307223-4c34ad73c8c0.json", scopes=scopes)

#gc = gspread.authorize(creds) # variable originalmente llamada client

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet_id  = '1bm90D0NjaSYtHsTDudzTFMqcWgVdkGqQMmpnFaiHUEA'

#%% CARGA DE HOJA DE CALCULO
# = = = = = = = = = = = = = =

sh = gc.open_by_key(spreadsheet_id)  #REEMPLAZAR AQUI CON EL NOMBRE DEL ARCHIVO
worksheet = gc.open_by_key(spreadsheet_id).sheet1  #REEMPLAZAR AQUI CON EL NOMBRE DEL ARCHIVO
rows = worksheet.get_all_values()
df = pd.DataFrame(rows[1:][:], columns = rows[0])


# Trato cadenas de texto vacías/representan no respuesta
# ---------------------------------------------------------}

# Convertir tipos de datos
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass  # Deja la columna como object si no se puede convertir

# Mostrar información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 341 entries, Marca temporal to Subir pauta firmada aquí
dtypes: datetime64[ns](2), float64(150), int64(6), object(183)
memory usage: 50.7+ KB


# DETECCION/FILTRADO DE COLUMNAS

## Procesamiento de columnas

In [5]:
#%% ELIMINO ESPACIOS EN BLANCOS DE LOS NOMBRES DE LAS COLUMNAS
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

columnas_con_espacios = [col for col in df.columns if col.strip() != col]

df.rename(columns={col: col.strip() for col in columnas_con_espacios}, inplace=True)


#%% COLOCO EN MINUSCULAS Y SIN ESPACIOS EN BLANCO INUTILES TODOS LOS VALORES
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

# Filtrar columnas de tipo objeto
# -------------------------------

columnas_obj = df.select_dtypes(include=['object']).columns


# Aplicar strip y lower a los valores de las columnas de tipo objeto
# -------------------------------------------------------------------

df[columnas_obj] = df[columnas_obj].apply(lambda x: x.str.strip().str.lower())

df.replace('', np.nan, inplace = True)
df.replace('-', np.nan, inplace = True)
df.replace({'no responde': np.nan}, inplace = True)


In [6]:
#%% PROCESAMIENTO DE COLUMNAS DE RESPUESTA CUATERNARIA
# = = = = = = = = = = = = = = = = = = = = = = = = = = =

texto = 'seleccione una opción'

columnas_C = df.filter(like='[C]')
columnas_CP = df.filter(like='[CP]')
columnas_NC = df.filter(like='[NC]')
columnas_NA = df.filter(like='[NA]')

# Reemplazar texto en las columnas filtradas
# ------------------------------------------

df[columnas_C.columns] = columnas_C.replace(texto, 'C')
df[columnas_CP.columns] = columnas_CP.replace(texto, 'CP')
df[columnas_NC.columns] = columnas_NC.replace(texto, 'NC')
df[columnas_NA.columns] = columnas_NA.replace(texto, 'NA')


#%% REEMPLAZO EN DATAFRAME ORIGINAl
# = = = = = = = = = = = = = = = = =

# Creo nuevas columnas, con pregunta original sin posible/s respuesta/s y agrego al df base
# -----------------------------------------------------------------------------------------

columnas_preguntas_C = []

for i in range(len(columnas_C.columns)):

    nuevo_nombre_col = columnas_C.columns[i].split(' [')[0]

    columnas_preguntas_C.append(nuevo_nombre_col)

    combined_series = pd.concat(
        [
            df[columnas_C.columns[i]],
            df[columnas_CP.columns[i]],
            df[columnas_NC.columns[i]],
            df[columnas_NA.columns[i]]
        ], axis=1).bfill(axis=1).iloc[:, 0]

    df[nuevo_nombre_col] = combined_series



#%% BORRO COLUMNAS [C] [CP] [NC] [NA]
# = = = = = = = = = = = = = = = = = =

lista_a_borrar = list(columnas_C.columns) + list(columnas_CP.columns) + list(columnas_NC.columns) + list(columnas_NA.columns)

df.drop(columns = lista_a_borrar, inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Columns: 152 entries, Marca temporal to Posee control de efluentes/manejo de los residuos/relevamiento de actividades críticas/utiliza transporte de sustancias peligrosas.
dtypes: datetime64[ns](2), float64(16), int64(6), object(128)
memory usage: 22.7+ KB


In [7]:
#%% FILTRO/DETECTO COLUMNAS QUE REQUIEREN DOCUMENTACIÓN ADJUNTA
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

cols_doc_adjunta = df.filter(like = 'djunt')
contar_doc_adj = cols_doc_adjunta[cols_doc_adjunta.columns].notna().sum()

print(f'Total entrevistados: {len(df)}. Adjuntaron documentación:')
print(contar_doc_adj.to_string())

Total entrevistados: 19. Adjuntaron documentación:
Si tiene implementado HACCP, adjunte el plan aquí    0
Adjunte la documentación aquí                        0


In [8]:
#%% TRABAJO CON COLUMNAS DE OBSERVACIONES
# = = = = = = = = = = = = = = = = = = = = =

cols_obs = df.filter(like = 'Obs')

# Reacomodo general
# -----------------

# Pruebo con una columna para detectar inconvenientes

# cols_obs[cols_obs.columns[3]].value_counts() # Me encuentro con problemas debido a transcipción manual. Corrijo

# Corrijo

cols_obs = cols_obs.applymap(lambda x: x.strip().lower() if pd.notna(x) else x)
cols_obs = cols_obs.applymap(lambda x: x.split(' (')[0] if pd.notna(x) else x)

In [9]:
#%% FILTRAR COLUMNAS DE RESPUESTA BINARIA (SI/NO)
# = = = = = = = = = = = = = = = = = = = = = = = = =

columnas_binarias = df.columns[df.isin(['si', 'no', np.nan]).all()]
#columnas_binarias = df.columns[df.isin(['si', 'no', '']).all()]


#%% CORRIJO COLUMNA PUNTUACION
# = = = = = = = = = = = = = = =

puntaje_maximo = 156

try:

    df['Puntuación'] = [int((int(x.split(' / ')[0]) / puntaje_maximo) * 100) for x in df['Puntuación']]

except:

    pass


#%% ELIMINO COLUMNAS QUE TIENEN TODOS NAN
# = = = = = = = = = = = = = = = = = = = =

# Lista de columnas innecesarias
columnas_inutiles = ['Marca temporal', 'Fecha', 'Empresa', 'Persona de contacto', 'Dirección',
                     'Teléfono', 'e-mail', 'e-mail (persona de contacto)', 'Nombre',
                     'Cargo', 'Departamento en el que trabaja',
                     'Nombre del líder del equipo de inocuidad',
                     'Nombre del Personal Calificado para Controles Preventivos']

# Detección de columnas que contienen solo NaN
deteccion_columnas_nan = df.columns[df.isin([np.nan]).all()]

# Unificar todas las columnas a eliminar en un solo conjunto para evitar duplicados
columnas_a_eliminar = set(columnas_inutiles).union(deteccion_columnas_nan)

# Eliminar las columnas
for col in columnas_a_eliminar:
    try:
        df.drop(columns=[col], inplace=True)
        print(f"Columna {col} eliminada exitosamente.")
    except KeyError:
        print(f"Advertencia: La columna {col} no existe en el DataFrame y no se pudo eliminar.")

df.info()

Columna Nombre del líder del equipo de inocuidad eliminada exitosamente.
Columna Si tiene implementado HACCP, adjunte el plan aquí eliminada exitosamente.
Columna Observaciones (Almacenaje y distribución) eliminada exitosamente.
Columna Marca temporal eliminada exitosamente.
Columna Observaciones (Equipamiento) eliminada exitosamente.
Columna Observaciones (Información General) eliminada exitosamente.
Columna Observaciones (Food Defense) eliminada exitosamente.
Columna Fecha eliminada exitosamente.
Columna e-mail eliminada exitosamente.
Columna Dirección eliminada exitosamente.
Columna Departamento en el que trabaja eliminada exitosamente.
Columna Nombre del Personal Calificado para Controles Preventivos eliminada exitosamente.
Columna Observaciones (Organización) eliminada exitosamente.
Columna e-mail (persona de contacto) eliminada exitosamente.
Columna Subir pauta firmada aquí eliminada exitosamente.
Columna Nombre eliminada exitosamente.
Columna Cargo eliminada exitosamente.
Column

# COMPATIBILIZAR COLUMNAS SEGUN SECCION

In [10]:
## COMPATIBILIZO COLUMNAS MICA CON COLUMNAS DE MI DF
# = = = = = = = = = = = = = = = = = = = = = = = = = =


s_df_org = [x for x in df.columns if x in s_organizacion]
s_df_info_gral = [x for x in df.columns if x in s_informacion_general]
s_df_persona = [x for x in df.columns if x in s_personal]
s_df_edificio_instalaciones = [x for x in df.columns if x in s_edificio_instalaciones]
s_df_equipamiento = [x for x in df.columns if x in s_equipamiento]
s_df_manejo_de_materiales = [x for x in df.columns if x in s_manejo_de_materiales]
s_df_instalaciones_sanitarias = [x for x in df.columns if x in s_instalaciones_sanitarias]
s_df_almacenaje_y_distribucion = [x for x in df.columns if x in s_almacenaje_y_distribucion]
s_df_control_de_laboratorio = [x for x in df.columns if x in s_control_de_laboratorio]
s_df_inocuidad = [x for x in df.columns if x in s_inocuidad]
s_df_food_defense = [x for x in df.columns if x in s_food_defense]
s_df_ambiente = [x for x in df.columns if x in s_ambiente]
s_df_responda_las_siguientes_preguntas = [x for x in df.columns if x in s_responda_las_siguientes_preguntas]
s_df_mapeo_de_cadena_de_suministro = [x for x in df.columns if x in s_mapeo_de_cadena_de_suministro]
s_df_perfil_proveedores_perfil = [x for x in df.columns if x in s_perfil_proveedores_perfil]
s_df_perfil_proveedores_mano_de_obra = [x for x in df.columns if x in s_perfil_proveedores_mano_de_obra]
s_df_agua = [x for x in df.columns if x in s_agua]
s_df_uso_de_pesticidas = [x for x in df.columns if x in s_uso_de_pesticidas]
s_df_residuos = [x for x in df.columns if x in s_residuos]
s_df_pauta_firmada = [x for x in df.columns if x in s_pauta_firmada]


#%% CREO DICCIONARIO PARA RELACIONAR SECCIONES CON SU LISTA DE PREGUNTAS (VALIDAS, DENTRO DEL DF) ASOCIADAS
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

relacion_secciones = {'Organización': s_df_org,
                      'Información General': s_df_info_gral,
                      'Persona': s_df_persona,
                      'Edificio e instalaciones': s_df_edificio_instalaciones,
                      'Equipamiento': s_df_equipamiento,
                      'Manejo de materiales': s_df_manejo_de_materiales,
                      'Instalaciones sanitarias': s_df_instalaciones_sanitarias,
                      'Almacenaje y Distribución': s_df_almacenaje_y_distribucion,
                      'Control de laboratorio': s_df_control_de_laboratorio,
                      'Inocuidad': s_df_inocuidad,
                      'Food Defense': s_df_food_defense,
                      'Ambiente': s_df_ambiente,
                      'Responda las siguientes preguntas': s_df_responda_las_siguientes_preguntas,
                      'Cadena de suministro': s_df_mapeo_de_cadena_de_suministro,
                      'Perfil de proovedores': s_df_perfil_proveedores_perfil,
                      'Mano de obra': s_df_perfil_proveedores_mano_de_obra,
                      'Sobre el agua de uso': s_df_agua,
                      'Uso de pesticidas': s_df_uso_de_pesticidas,
                      'Residuos': s_df_residuos}



# GENERACION ARCHIVO DE SALIDA

In [11]:
# Agregar una nueva hoja para los datos procesados
# ------------------------------------------------

df.replace(np.nan, '', inplace = True)

new_sheet_name = 'Datos Procesados'

try:
    sh.add_worksheet(title=new_sheet_name, rows=str(df.shape[0] + 1), cols=str(df.shape[1]))
except gspread.exceptions.APIError:

    # Si la hoja ya existe, podemos optar por obtenerla y limpiarla
    # -------------------------------------------------------------

    worksheet_new = sh.worksheet(new_sheet_name)
    worksheet_new.clear()
else:
    worksheet_new = sh.worksheet(new_sheet_name)

# Volcar los datos procesados en la nueva hoja
# -------------------------------------------

worksheet_new.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1bm90D0NjaSYtHsTDudzTFMqcWgVdkGqQMmpnFaiHUEA',
 'updatedRange': "'Datos Procesados'!A1:DX20",
 'updatedRows': 20,
 'updatedColumns': 128,
 'updatedCells': 2560}

In [12]:
# Configurar la autenticación con Google Sheets


# Tu DataFrame procesado
#processed_data = df  # Reemplaza con tu DataFrame de datos procesados

# Convertir DataFrame a lista de listas y actualizar la hoja
#sheet.update([processed_data.columns.values.tolist()] + processed_data.values.tolist())

# CREACION DE DASHBORAD

## Salida en entorno de Notebook

In [13]:
'''
#%% CREAR LA APP DE DASH
## = = = = = = = = = = = =

app = dash.Dash(__name__)


# Layout del dashboard
# --------------------

app.layout = html.Div([
    html.H1("Encuestas - Plantas de Empaque"),
    dcc.Dropdown(
        id='section-dropdown',
        options=[{'label': sec, 'value': sec} for sec in relacion_secciones.keys()],
        value=list(relacion_secciones.keys())[0]
    ),
    html.Div(id='graphs-container')
])


# Callback para actualizar los gráficos según la sección seleccionada
# -------------------------------------------------------------------

@app.callback(
    Output('graphs-container', 'children'),
    [Input('section-dropdown', 'value')]
)

def update_graphs(selected_section):

    columnas = relacion_secciones[selected_section]

    lista_temp = [x for x in columnas if x in columnas_preguntas_C]
    lista_torta = [x for x in columnas if x not in columnas_preguntas_C]

    figuras = []

    if len(lista_temp) > 0:
        figuras.extend(grf_subplots_barras(df, lista_temp))

    if len(lista_torta) > 0:
        figuras.extend(grf_torta(df, lista_torta))

    graph_divs = []

    for i, fig in enumerate(figuras):

        graph_divs.append(dcc.Graph(figure=fig, style={'width': '100%', 'display': 'inline-block', 'padding': '10px'}))

        if (i+1) % 2 == 0:

            graph_divs.append(html.Br())  # Agregar un salto de línea después de cada dos gráficos

    return graph_divs


# Ejecucion de la app
# -------------------

if __name__ == '__main__':
    app.run_server(debug=True, mode='internal')
''';

## Salida a nueva pestaña del navegador

In [14]:
'''
#%% CREACION DE LA APP DE DASH UTILIZANDO JUPYTERDASH
# = = = = = = = = = = = = = = = = = = = = = = = = = = =

app = JupyterDash(__name__)

# Layout del dashboard
# --------------------

app.layout = html.Div([
    html.H1("Encuestas - Empaque"),
    dcc.Dropdown(
        id='section-dropdown',
        options=[{'label': sec, 'value': sec} for sec in relacion_secciones.keys()],
        value=list(relacion_secciones.keys())[0]
    ),
    html.Div(id='graphs-container')
])

# Callback para actualizar los gráficos según la sección seleccionada
# -------------------------------------------------------------------

@app.callback(
    Output('graphs-container', 'children'),
    [Input('section-dropdown', 'value')]
)


def update_graphs(selected_section):

    columnas = relacion_secciones[selected_section]

    lista_temp = [x for x in columnas if x in columnas_preguntas_C]
    lista_torta = [x for x in columnas if x not in columnas_preguntas_C]

    figuras = []

    if len(lista_temp) > 0:
        figuras.extend(grf_subplots_barras(df, lista_temp))

    if len(lista_torta) > 0:
        figuras.extend(grf_torta(df, lista_torta))

    graph_divs = []

    for i, fig in enumerate(figuras):

        graph_divs.append(dcc.Graph(figure=fig, style={'width': '100%', 'display': 'inline-block', 'padding': '10px'}))

        if (i+1) % 2 == 0:

            graph_divs.append(html.Br())  # Agregar un salto de línea después de cada dos gráficos

    return graph_divs

# Ejecutar la app en modo externo
# -------------------------------

#app.run_server(mode='external')

# Abrir automáticamente una nueva pestaña del navegador al correr la notebook
# ---------------------------------------------------------------------------

port = 8050 # or simply open on the default `8050` port

def open_browser():
	webbrowser.open_new("http://localhost:{}".format(port))

if __name__ == '__main__':
    Timer(1, open_browser).start();
    app.run_server(debug=True, port=port)
''';

https://github.com/ebmdatalab/openpath-dash/commit/fe7b1aae5743ae95e48d1cac4a0219aad14bbab3;
https://community.plotly.com/t/auto-open-browser-window-with-dash/31948/2;

In [15]:
print("forkin"+"me")

#!pip install selenium pike validators

#!python colabctl.py forkinme 300

forkinme
